# ERA5 post-processed daily statistics on single levels from 1940 to present

The data is stored in these directories, each one has the daily data for one month of the corresponding year in a zip file.

```bash
data  
└── ERA5  
    └── daily  
        ├── 2020  
        │   ├── 01  
        │   ├── 02  
        │   ├── 03  
        │   ├── 04  
        │   ├── 05  
        │   ├── 06  
        │   ├── 07  
        │   ├── 08  
        │   ├── 09  
        │   ├── 10  
        │   ├── 11  
        │   └── 12  
        └── 2021  
            ├── 01  
            ├── 02  
            ├── 03  
            ├── 04  
            ├── 05  
            ├── 06  
            ├── 07  
            ├── 08  
            ├── 09  
            ├── 10  
            ├── 11  
            └── 12

```


In [37]:
import duckdb
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import glob
import zipfile
import os
from functools import reduce

pd.set_option('display.max_columns', None)

## Unzipping

In [17]:
years = ["2020"]
months = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12"
]

for year in years:
    for month in months:
        zip_file = glob.glob(f"data/ERA5/daily/{year}/{month}/*.zip")[0]
        print(f"Unzipping: {zip_file} in data/ERA5/daily/{year}/{month}/")

        extract_dir = os.path.dirname(zip_file)  # same folder as zip file
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)

Unzipping: data/ERA5/daily/2020/01/e9730e7225e3c3f6a5cd36c5084e46f4.zip in data/ERA5/daily/2020/01/
Unzipping: data/ERA5/daily/2020/02/d8c7eb1ccdc5522bf344b8e7e76e91f7.zip in data/ERA5/daily/2020/02/
Unzipping: data/ERA5/daily/2020/03/3929c34888d3446f532edfe4c5a1c80a.zip in data/ERA5/daily/2020/03/
Unzipping: data/ERA5/daily/2020/04/ac3f178e902ef4c5c80cb99317e48a74.zip in data/ERA5/daily/2020/04/
Unzipping: data/ERA5/daily/2020/05/cbc7fffb19692acbfa9d0813f83d4e75.zip in data/ERA5/daily/2020/05/
Unzipping: data/ERA5/daily/2020/06/49237642dab5899c40832456c10a180d.zip in data/ERA5/daily/2020/06/
Unzipping: data/ERA5/daily/2020/07/ac8b843f4d4ed3e689d87edb8b6fa6ec.zip in data/ERA5/daily/2020/07/
Unzipping: data/ERA5/daily/2020/08/873d18dd569bf3293ea2a8f0a86e6647.zip in data/ERA5/daily/2020/08/
Unzipping: data/ERA5/daily/2020/09/f3ab3148e45d55ffae27569b03785a89.zip in data/ERA5/daily/2020/09/
Unzipping: data/ERA5/daily/2020/10/65eb3d975ca67c373f9df116ec91fa19.zip in data/ERA5/daily/2020/10/


## DuckDB database

### Create table

In [30]:
con = duckdb.connect('data/iNaturalist.duckdb')
print("connected!")

connected!


In [44]:
con.execute("""
CREATE TABLE IF NOT EXISTS ERA5_Daily (
    valid_time DATE,
    latitude DOUBLE,
    longitude DOUBLE,
    u10 DOUBLE,
    number INTEGER,
    v10 DOUBLE,
    d2m DOUBLE,
    t2m DOUBLE,
    msl DOUBLE,
    mwd DOUBLE,
    mwp DOUBLE,
    sst DOUBLE,
    swh DOUBLE,
    sp DOUBLE,
    tp DOUBLE
);
""")
print("Table ERA5_Daily created successfully!")

Table ERA5_Daily created successfully!


In [45]:
# List all tables
tables = con.execute("SHOW TABLES").fetchdf()
print(tables)

          name
0   ERA5_Daily
1         grid
2     grid_vss
3  iNaturalist


In [46]:
query = """
SELECT *
FROM ERA5_Daily
LIMIT 5
"""
df = con.execute(query).df()
df

,valid_time,latitude,longitude,u10,number,v10,d2m,t2m,msl,mwd,mwp,sst,swh,sp,tp


### Insert data

In [47]:
years = ["2020"]
months = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12"
]

for year in years:
    for month in tqdm(months):
        files = glob.glob(f"data/ERA5/daily/{year}/{month}/*.nc")

        dfs = []
        for f in files:
            ds = xr.open_dataset(f)
            df = ds.to_dataframe().reset_index()
            dfs.append(df)

        # Assuming your list of DataFrames is called `dfs`
        df_merged = reduce(
            lambda left, right: pd.merge(
                left, right, on=['valid_time', 'latitude', 'longitude', 'number'], how='outer'
            ),
            dfs
        )

        con.execute("INSERT INTO ERA5_Daily SELECT * FROM df_merged")
        print(f"Inserted {len(df_merged)} rows into ERA5_Daily for {year}-{month}.")  

  8%|▊         | 1/12 [02:42<29:46, 162.41s/it]

Inserted 32185440 rows into ERA5_Daily for 2020-01.


 17%|█▋        | 2/12 [05:15<26:07, 156.71s/it]

Inserted 30108960 rows into ERA5_Daily for 2020-02.


 25%|██▌       | 3/12 [07:57<23:55, 159.50s/it]

...

Inserted 32185440 rows into ERA5_Daily for 2020-10.


 92%|█████████▏| 11/12 [29:12<02:38, 158.08s/it]

Inserted 31147200 rows into ERA5_Daily for 2020-11.


100%|██████████| 12/12 [31:53<00:00, 159.49s/it]

Inserted 32185440 rows into ERA5_Daily for 2020-12.


In [48]:
# con.execute("DROP TABLE IF EXISTS ERA5_Daily;")
# print("Table ERA5_Daily dropped (if it existed).")

## Grid coordinates

In [ ]:
# Get all unique latitude-longitude pairs
query = """
SELECT DISTINCT latitude, longitude
FROM ERA5_Daily
ORDER BY latitude, longitude
"""
df = con.execute(query).df()

# Add a unique ID column
df["grid_id"] = range(len(df))

# Create the grid table in DuckDB
con.execute("CREATE TABLE IF NOT EXISTS ERA5_grid AS SELECT * FROM df")

print("Table 'ERA5_grid' created successfully!")


In [ ]:
# con.execute("DROP TABLE IF EXISTS era5_grid;")

In [59]:
query = """
SELECT *
FROM ERA5_Daily
WHERE valid_time = DATE('2020-01-01')
"""
df = con.execute(query).df()
df

,valid_time,latitude,longitude,u10,number,v10,d2m,t2m,msl,mwd,mwp,sst,swh,sp,tp
0,2020-01-01,-90.0,0.00,0.532273,0,0.025590,242.739258,246.011063,99628.218750,NaN,NaN,NaN,NaN,68844.367188,4.569689e-07
1,2020-01-01,-90.0,0.25,0.532273,0,0.025590,242.739258,246.011063,99628.218750,NaN,NaN,NaN,NaN,68844.367188,4.569689e-07
2,2020-01-01,-90.0,0.50,0.532273,0,0.025590,242.739258,246.011063,99628.218750,NaN,NaN,NaN,NaN,68844.367188,4.569689e-07
3,2020-01-01,-90.0,0.75,0.532273,0,0.025590,242.739258,246.011063,99628.218750,NaN,NaN,NaN,NaN,68844.367188,4.569689e-07
4,2020-01-01,-90.0,1.00,0.532273,0,0.025590,242.739258,246.011063,99628.218750,NaN,NaN,NaN,NaN,68844.367188,4.569689e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038235,2020-01-01,90.0,358.75,-0.547764,0,0.120316,242.743652,245.892822,100475.882812,NaN,NaN,271.460419,NaN,100475.250000,7.112821e-06
1038236,2020-01-01,90.0,359.00,-0.547764,0,0.120316,242.743652,245.892822,100475.882812,NaN,NaN,271.460419,NaN,100475.250000,7.112821e-06
1038237,2020-01-01,90.0,359.25,-0.547764,0,0.120316,242.743652,245.892822,100475.882812,NaN,NaN,271.460419,NaN,100475.250000,7.112821e-06
1038238,2020-01-01,90.0,359.50,-0.547764,0,0.120316,242.743652,245.892822,100475.882812,NaN,NaN,271.460419,NaN,100475.250000,7.112821e-06


In [60]:
query = """
SELECT e.*, g.grid_id
FROM ERA5_Daily AS e
    INNER JOIN grid AS g
    ON e.latitude = g.latitude AND e.longitude = g.longitude
WHERE valid_time = DATE('2020-01-01')
"""
df = con.execute(query).df()
df

,valid_time,latitude,longitude,u10,number,v10,d2m,t2m,msl,mwd,mwp,sst,swh,sp,tp,grid_id
0,2020-01-01,20.0,320.00,-4.486241,0,-1.625940,291.982025,296.448151,101751.554688,280.737244,8.984914,297.897491,1.710659,101753.375000,0.000017,634880
1,2020-01-01,20.0,320.25,-4.649815,0,-1.403894,292.162750,296.293915,101752.500000,NaN,NaN,297.849579,NaN,101756.000000,0.000018,634881
2,2020-01-01,20.0,320.50,-4.797073,0,-1.139897,292.298737,296.170471,101754.312500,310.235901,8.942210,297.821167,1.700873,101758.539062,0.000023,634882
3,2020-01-01,20.0,320.75,-4.926915,0,-0.864303,292.367767,296.068329,101756.781250,NaN,NaN,297.784546,NaN,101759.250000,0.000033,634883
4,2020-01-01,20.0,321.00,-5.025222,0,-0.626225,292.348633,295.984100,101759.156250,354.969635,8.906738,297.721802,1.693040,101758.250000,0.000037,634884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038235,2020-01-01,-47.5,238.75,6.014695,0,8.232133,277.388824,281.943451,102433.398438,NaN,NaN,283.122192,NaN,102433.539062,0.000009,245755
1038236,2020-01-01,-47.5,239.00,6.182135,0,8.417232,277.523346,281.958160,102403.132812,203.624939,7.895751,283.206757,2.893896,102405.789062,0.000008,245756
1038237,2020-01-01,-47.5,239.25,6.349859,0,8.595495,277.661133,281.986969,102372.945312,NaN,NaN,283.326385,NaN,102374.539062,0.000006,245757
1038238,2020-01-01,-47.5,239.50,6.502854,0,8.775793,277.776367,281.995697,102341.539062,204.433533,7.950001,283.338104,3.047725,102340.250000,0.000007,245758
